In [2]:
import datetime
import pandas as pd
import numpy as np
import time
from dateutil.parser import parse
from new_asin import new_asin
from origin_to_sql import origin_to_sql, origin_sql_total
from origin_to_skusale import origin_to_skusale
from check import check
from sale_to_sql import sale_sql_temp, sale_sql_total
from dataprocess import read
import pandas as pd
from database_read import read
from sqlalchemy import create_engine
import re
import warnings
warnings.filterwarnings("ignore")

In [3]:
origin = read('after_sales','new_after_salesv2_2021')

In [4]:
AdDate = datetime.date(2021,1, 1)
ENDDate = datetime.date(2021,4, 1)
origin['登记日期'] = pd.to_datetime(origin['登记日期'])
origin_2021 = origin[(origin['登记日期'] >= datetime.datetime.combine(AdDate, datetime.datetime.min.time()))&(origin['登记日期'] <= datetime.datetime.combine(ENDDate, datetime.datetime.min.time()))]

# origin_2021[origin_2021['顾客反馈'].isnull()]

In [5]:
#找出有总退款》50的ab箱订单
origin_2021two= pd.pivot_table(origin_2021,index = '订单号',aggfunc = { 'Refund':'count'} )
origin_2021two = origin_2021two[origin_2021two['Refund']>1]
origin_2021two = origin_2021two.reset_index()

origin_2021 = origin_2021[origin_2021['订单号'].isin(origin_2021two['订单号'])]

origin_big50 = origin_2021[origin_2021['Refund'] >50]
print(origin_big50) 
origin_big50.to_excel(excel_writer="D:/work/origin_big50.xlsx", index=False,encoding='utf-8')

,id,登记日期,更新日期,登记人,店铺,订单号,SKU,序号,订单状态,顾客反馈,客服操作,Refund,备注,是否上传图片


In [7]:
#即登记在2022年有登记在2021年的订单
# notin = origin_2022[~origin_2022['订单号'].isin(origin_2021['订单号'])]
qwe = origin_2021[origin_2021['备注']=='取消']
qwe1 = origin_2021[origin_2021['订单号'].isin(qwe['订单号'])]
qwe1.to_excel(excel_writer="D:/work/21年ab箱取消1.xlsx", index=False,encoding='utf-8')
qwe1


,id,登记日期,更新日期,登记人,店铺,订单号,SKU,序号,订单状态,顾客反馈,客服操作,Refund,备注,是否上传图片


In [9]:
AdDate = datetime.date(2021,1, 1)
ENDDate = datetime.date(2021,4, 1)
origin['登记日期'] = pd.to_datetime(origin['登记日期'])
origin_2021 = origin[(origin['登记日期'] >= datetime.datetime.combine(AdDate, datetime.datetime.min.time()))&(origin['登记日期'] <= datetime.datetime.combine(ENDDate, datetime.datetime.min.time()))]

guke= origin_2021[origin_2021['顾客反馈']== '']
guke

,id,登记日期,更新日期,登记人,店铺,订单号,SKU,序号,订单状态,顾客反馈,客服操作,Refund,备注,是否上传图片
96,100,2021-01-02,2022-05-11,方婷,amazon赫曼,112-2160252-6702603,USAN1011322-3,1,已解决,,1-2-0,20.00,买家购买前没有领取优惠券,未上传
1912,1951,2021-01-21,2021-01-21,杨豪狄,wayfair信盒,CS300802332,HOG100864-03,1,已解决,,3-1-0$HOG100865-03num1,0.00,None,未检查
1913,1952,2021-01-21,2021-01-21,杨豪狄,wayfair信盒,CS302475469,USAN1011085-2,1,已解决,,3-1-0$HOG100865-01num1,0.00,None,未检查
2231,2276,2021-01-25,2022-05-17,王俊杰,amazon信盒,114-9112542-3101042,USAN1018441-2,1,已解决,,1-2-0,190.03,记录找不到了,未检查
2904,2974,2021-02-01,2022-05-16,敖睿洁,amazon优瑞斯特,113-8920052-0910603,USAN1011334-1,1,已解决,,1-2-0,0.00,此条无效,未检查
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9786,9999,2021-04-01,2021-04-01,王俊杰,amazon信盒,114-7598106-7351405,HOG101439-01,1,已解决,,1-2-0,179.98,None,未检查
9787,10000,2021-04-01,2021-04-01,徐志炜,amazon信盒,114-7640678-3541840,HOG100555-02,1,已解决,,1-2-0,70.00,None,未检查
9788,10001,2021-04-01,2021-04-01,徐欢,amazon信盒,114-8579186-1806666,USAN1018441-1,1,已解决,,1-2-0,189.98,None,未检查
9796,10009,2021-04-01,2021-04-01,王奉海,amazon信盒,114-9686388-4092244,USAN1019552-8,1,已解决,,1-2-0,30.00,None,未检查


In [10]:
guke.to_excel(excel_writer="D:/work/guke.xlsx", index=False,encoding='utf-8')

# 统计sku月度数据

In [118]:
inventory = read('inventory','自发货源数据new1')

In [119]:
AdDate = datetime.date(2021,1, 1)
ENDDate = datetime.date(2022,5, 1)
inventory['订单时间'] = pd.to_datetime(inventory['订单时间'])
# inventory['补寄sku订单时间'] = pd.to_datetime(inventory['补寄sku订单时间'])
# inventory['补寄配件订单时间'] = pd.to_datetime(inventory['补寄配件订单时间'])
inventory = inventory[["公司SKU","补寄sku订单时间","补寄配件订单时间","账号","补寄sku数量","补寄配件次数"]]
inventory = inventory[(inventory['订单时间'] >= datetime.datetime.combine(AdDate, datetime.datetime.min.time()))&(inventory['订单时间'] < datetime.datetime.combine(ENDDate, datetime.datetime.min.time()))]
# inventory = inventory[(inventory['补寄配件订单时间'] >= datetime.datetime.combine(AdDate, datetime.datetime.min.time()))|(inventory['补寄配件订单时间'] < datetime.datetime.combine(ENDDate, datetime.datetime.min.time()))&(inventory['补寄sku订单时间'] >= datetime.datetime.combine(AdDate, datetime.datetime.min.time()))&(inventory['补寄sku订单时间'] < datetime.datetime.combine(ENDDate, datetime.datetime.min.time()))]


inventory

,公司SKU,补寄sku订单时间,补寄配件订单时间,账号,补寄sku数量,补寄配件次数
64,USAN1015827-1,NaT,2021-02-10 23:12:00,amazon宫本,NaN,1.0
406,USAN1017103-1,NaT,2021-04-13 16:21:00,amazon赫曼,NaN,1.0
1370,HOG101439-01,NaT,2021-05-05 10:47:00,amazon信盒,NaN,1.0
2305,USAN1019789-5,NaT,2021-04-22 17:27:00,amazon尚铭,NaN,1.0
2311,USAN1016483-1,NaT,2021-04-29 14:44:00,amazon优瑞斯特,NaN,1.0
...,...,...,...,...,...,...
1691673,USAN1022706-4,NaT,2022-04-27 12:28:00,shopifynextfur,NaN,2.0
1691685,USAN1022780-4,NaT,2022-03-31 10:17:00,shopifynextfur,NaN,1.0
1691736,USAN1022780-4,NaT,2022-01-25 10:56:00,shopifynextfur,NaN,1.0
1691821,USAN1022780-1,NaT,2022-02-25 15:19:00,shopifynextfur,NaN,1.0


In [120]:
# inventory['year'] = inventory['补寄sku订单时间'].dt.year 
# inventory['month'] = inventory['补寄sku订单时间'].dt.month
# inventory['year1'] = inventory['补寄配件订单时间'].dt.year 
# inventory['month1'] = inventory['补寄配件订单时间'].dt.month

In [122]:
inventory['year'] = inventory['订单时间'].dt.year 
inventory['month'] = inventory['订单时间'].dt.month
inventory = inventory.pivot_table(index=['公司SKU','year','month','year1','month1','账号'], values=['补寄sku数量',"补寄配件次数"], aggfunc={'补寄sku数量':'sum',"补寄配件次数":"sum"})
inventory = inventory.reset_index()
inventory

,公司SKU,year,month,year1,month1,账号,补寄sku数量,补寄配件次数
0,HOG102201-05,2021.0,1.0,2021,1,amazon优瑞斯特,1.0,2.0
1,HOG102201-06,2021.0,5.0,2021,5,amazon优瑞斯特,1.0,1.0
2,PTG103482-3,2021.0,5.0,2021,3,amazon优瑞斯特,1.0,1.0
3,USAN1011082-5,2021.0,2.0,2021,3,amazon优瑞斯特,1.0,1.0
4,USAN1011315-3,2021.0,3.0,2021,1,amazon信盒,1.0,1.0
5,USAN1011320-1,2021.0,5.0,2021,4,amazon赫曼,1.0,1.0
6,USAN1011338-2,2021.0,4.0,2021,3,amazon赫曼,1.0,1.0
7,USAN1011338-3,2021.0,1.0,2021,1,amazon优瑞斯特,1.0,1.0
8,USAN1015161-2,2020.0,11.0,2021,1,amazon赫曼,1.0,1.0
9,USAN1015161-3,2021.0,3.0,2021,2,amazon赫曼,1.0,2.0


In [124]:
inventory.to_excel(excel_writer="D:/work/invnetory_buji1.xlsx", index=False,encoding='utf-8')

In [115]:
inventory = read('inventory','自发货源数据new1')

AdDate = datetime.date(2022,4, 1)
ENDDate = datetime.date(2022,5, 1)
inventory['订单时间'] = pd.to_datetime(inventory['订单时间'])
inventory['补寄sku订单时间'] = pd.to_datetime(inventory['补寄sku订单时间'])
inventory['补寄配件订单时间'] = pd.to_datetime(inventory['补寄配件订单时间'])
inventory = inventory[["公司SKU","订单时间","补寄sku订单时间","补寄配件订单时间","账号",'数量']]
inventory = inventory[(inventory['订单时间'] >= datetime.datetime.combine(AdDate, datetime.datetime.min.time()))&(inventory['订单时间'] < datetime.datetime.combine(ENDDate, datetime.datetime.min.time()))]
inventory

,公司SKU,订单时间,补寄sku订单时间,补寄配件订单时间,账号,数量
1003256,USAN1025742-1,2022-04-23 00:31:00,NaT,NaT,amazon信盒,1.0
1003257,USAN1029450-1,2022-04-23 02:32:00,NaT,NaT,amazon驰甬,10.0
1003258,USAN1034749-3,2022-04-23 07:45:00,NaT,NaT,amazon驰甬,1.0
1003259,USAN1023786-3,2022-04-23 02:31:00,NaT,NaT,amazon赫曼,1.0
1003260,USAN1023893-2,2022-04-23 02:32:00,NaT,NaT,amazon森月,1.0
...,...,...,...,...,...,...
1695175,USAN1034651-1,2022-04-03 15:01:00,NaT,NaT,shopifynextfur,1.0
1695176,USAN1034653-6,2022-04-03 10:01:00,NaT,NaT,shopifynextfur,1.0
1695177,USAN1023809-5,2022-04-03 08:01:00,NaT,NaT,shopifynextfur,1.0
1695178,USAN1011338-1,2022-04-03 10:01:00,NaT,NaT,shopifynextfur,1.0


In [116]:
inventory['year'] = inventory['订单时间'].dt.year 
inventory['month'] = inventory['订单时间'].dt.month
inventory = inventory.pivot_table(index=['公司SKU','year','month','账号'], values=['数量'], aggfunc='sum')
inventory = inventory.reset_index()
inventory[inventory['month'] == 5]
# inventory

,公司SKU,订单时间,补寄sku订单时间,补寄配件订单时间,账号,数量,year,month


In [57]:
inventory.to_excel(excel_writer="D:/work/invnetory2.xlsx", index=False,encoding='utf-8')

# 统计sku补寄月度数据

In [3]:
inventory1 = read('inventory','自发货源数据new1')

# import math
# for i in range(len(inventory1)):
#     if math.isnan(inventory1.loc[i,'数量']) and not math.isnan(inventory1.loc[i,'补寄数量']):
#         inventory1.loc[i,'数量'] = inventory1.loc[i,'补寄数量']

In [47]:
inventory = inventory1.copy()


In [48]:
inventory.loc[inventory['补寄配件次数'].isnull(),'补寄配件次数'] = 0
inventory.loc[inventory['补寄sku数量'].isnull(),'补寄sku数量'] = 0

In [49]:
inventory['补寄数量'] = inventory['补寄sku数量']+inventory['补寄配件次数']
inventory = inventory[inventory['补寄数量']>0]

In [51]:

AdDate = datetime.date(2021,1, 1)
ENDDate = datetime.date(2022,5, 1)
# inventory['发货时间'] = pd.to_datetime(inventory['发货时间'])
inventory['补寄sku订单时间'] = pd.to_datetime(inventory['补寄sku订单时间'])
inventory['补寄配件订单时间'] = pd.to_datetime(inventory['补寄配件订单时间'])
inventory = inventory[["公司SKU","补寄配件订单时间",'补寄sku订单时间',"账号",'补寄数量']]
inventory = inventory[((inventory['补寄sku订单时间'] >= datetime.datetime.combine(AdDate, datetime.datetime.min.time()))&(inventory['补寄sku订单时间'] < datetime.datetime.combine(ENDDate, datetime.datetime.min.time())))|((inventory['补寄配件订单时间'] >= datetime.datetime.combine(AdDate, datetime.datetime.min.time()))&(inventory['补寄配件订单时间'] < datetime.datetime.combine(ENDDate, datetime.datetime.min.time())))]
# inventory.loc[inventory['补寄配件订单时间'].isnull()& ~inventory['补寄sku订单时间'].isnull(),'补寄配件订单时间'] = inventory['补寄sku订单时间']
# inventory.drop(['补寄sku订单时间'],axis = 1)


In [52]:
inventory.loc[inventory['补寄配件订单时间'].isnull()& ~inventory['补寄sku订单时间'].isnull(),'补寄配件订单时间'] = inventory['补寄sku订单时间']
inventory['year'] = inventory['补寄配件订单时间'].dt.year 
inventory['month'] = inventory['补寄配件订单时间'].dt.month
inventory = inventory.pivot_table(index=['公司SKU','year','month','账号'], values=['补寄数量'], aggfunc="sum")
inventory = inventory.reset_index()


In [36]:
#这个出来总数较少
inventory.to_excel(excel_writer="D:/work/invnetorybuji22.xlsx", index=False,encoding='utf-8')

In [37]:

originzifa = pd.DataFrame()
for x in range(1,14):
    origin_zi = pd.read_table(r"D:/work/自发货订单管理源数据/系统数据/新数据_周/%s.txt" % x )
    origin_zifa = pd.DataFrame(origin_zi, columns=['订单号', '账号', '平台SKU','公司SKU', '数量', '包裹号','订单类型', '订单状态', '创建时间'])
    originzifa = originzifa.append(origin_zifa)
    originzifa = originzifa.reset_index(drop=True)
originzifa = originzifa.rename(columns={'订单号': '平台订单号','创建时间': '订单时间'})#统一列名
originzifa['订单时间'] =pd.to_datetime(originzifa['订单时间'])
originzifa =originzifa[~originzifa["订单状态"].str.contains('已取消')]  #去掉已取消的订单
originzifa.head(2)
print('新系统完成')
#存在正常订单取消改发补寄订单的情况，所以将这些订单改为正常订单
originzifa_list = originzifa.pivot_table(index=['平台订单号'], values=['数量'], aggfunc='count')
originzifa_list= originzifa_list.reset_index()
originzifa_list =originzifa_list[originzifa_list['数量'] == 1] #将单个正常订单和单个补寄订单的订单号取出
#在整个数据中再到单个正常订单和单个补寄订单的记录，筛选出其中的补寄订单
originzifa_order = originzifa[originzifa["平台订单号"].isin(originzifa_list["平台订单号"])]
originzifa_listpivot = pd.merge(originzifa_order[['平台订单号','订单类型']],originzifa_list,on = '平台订单号')
buji = originzifa_listpivot[originzifa_listpivot['订单类型'] == '补寄订单']
# index = originzifa.loc[originzifa['平台订单号'].isin(buji['平台订单号']),:].index
#将筛选出来的补寄订单改为正常订单
originzifa.loc[originzifa['平台订单号'].isin(buji['平台订单号']),'订单类型'] = '正常订单'
originzifa =originzifa[~originzifa['平台订单号'].isnull()]

#温馨提示一下所用字段可能需要增加和改变 比如平台SKU
# 后续因为需要统计退货率以及地区分区 所以需要把邮编拿进来 这一块暂时不考虑（因为我还没想好拿哪些字段）
#暂时你需要哪些字段你就拿哪些字段 这些是我当时认为要拿的
print(originzifa.head(2))

#以下是旧系统数据 我们在2021.10之前用的是另一个系统 这些旧数据旧直接读好了 字段和新的不一样 你可以像我这样分开处理再合并
# 也可以一起改名后统一处理
originzifa2021 = pd.DataFrame()
# 过往数据（旧系统）
from pathlib import Path
for x in range(1, 27):
    origin_2021 = pd.read_csv(r'D:/work/自发货订单管理源数据/系统数据/旧数据/旧%s.csv' % x)
    #     print(origin_2021)  #将所有的订单标准来源表，在之后的查看中可以明确知道那个订单来自那个表
    #     origin_2021['表'] = x
    origin_zifa2021 = pd.DataFrame(origin_2021,
                                   columns=['订单号', '账号', '原始sku', 'sku', '数量', '包裹号', '订单类型', '订单状态', '发货时间'])

    originzifa2021 = originzifa2021.append(origin_zifa2021)
    originzifa2021 = originzifa2021.reset_index(drop=True)
originzifa2021 = originzifa2021.rename(columns={'sku': '公司SKU', '原始sku': '平台SKU', '订单号': '平台订单号', '发货时间': '订单时间'})
# 防止重复读取
originzifa2021 = originzifa2021.drop_duplicates()

# print(originzifa2021.head(2))
print('旧系统完成')

#旧数据的包裹号会有出现-1,-2的形式，所以要将正确的包裹号提取出来。
originzifa2021['包裹号'] = originzifa2021['包裹号'].apply(lambda x:x.split("-")[0])

originzifa = pd.concat([originzifa, originzifa2021], axis=0)
originzifa = originzifa.reset_index(drop=True)
print(originzifa.head(2))
print('拼接完成')

#新旧系统对于店铺名命名方式也不同所以写个正则翻译一下店铺名
#后续我们自己的数据都会以这个命名方式为准
fanyistr = {
    "yourui Walmart US": 'walmart优瑞斯特',
    "heman Walmart US": 'walmart赫曼',
    "xinhemaoyi Walmart US": 'walmart信盒',
    "gongben Walmart US": 'walmart宫本',
    "xinhemaoyi wayfair": 'wayfair信盒',
    "weilu wayfair": 'wayfair维禄',
    "zhirun2021": 'ebay治润',
    "yaqinsale11": 'ebay雅秦',
    "linglang2021": 'ebay玲琅',
    "linglanglv21": 'ebay玲琅',
    "简砾": 'amazon简砾',
    "shangming": 'amazon尚铭',
    "jianli": 'amazon简砾',
    "heman": 'amazon赫曼',
    "赫曼": 'amazon赫曼',
    "Youruisite": 'amazon优瑞斯特',
    "信盒贸易": 'amazon信盒',
    "玲琅": 'amazon玲琅',
    "Central Power International Limited": 'amazoncpower',
    "维禄": 'amazon维禄',
    "宫本": 'amazon宫本',
    "xinhemaoyi": 'amazon信盒',
    "优瑞斯特": 'amazon优瑞斯特',
    "森月": 'amazon森月',
    "驰甬": 'amazon驰甬',
    "尚铭": 'amazon尚铭',
    "damaiwang": 'amazon哒唛旺',
    "哒唛旺": 'amazon哒唛旺',
    "bulubulu": 'amazon卟噜卟噜',
    "治润": 'amazon治润',
    "启珊": 'amazon启珊',
    "卟噜卟噜": 'amazon卟噜卟噜',
    "旗辰": 'amazon旗辰',
    "senyue": 'amazon森月',
    "NEXTFUR LLC": 'shopifynextfur',
    "赛迦曼": 'amazon赛迦曼',
    "gongben": 'amazon宫本',
    "weilu": 'amazon维禄',
    "NEXTFUR": 'shopifynextfur',
}


def tran(my_str, rep):
    rep = dict((re.escape(k), v) for k, v in rep.items())
    pattern = re.compile("|".join(rep.keys()))
    my_str = pattern.sub(lambda m: rep[re.escape(m.group(0))], my_str)
    return my_str

# print(type(originzifa['账号'] ))
# 翻译
originzifa['账号'] = originzifa['账号'].apply(lambda x: tran(x, fanyistr))
#之所以加平台是为了优化平台订单号 你可以考虑把平台和账号属性合二为一 节约储存空间
#这是一个很重要的坑 早期的订单号很混乱（旧系统） 同一单下有多个产品时
#比如第一单会写原订单号
#第二单会写-1 第三单有时候写-1-1 有时候写-2
#还原为原订单号 方便查找检索
originzifa['平台'] = 0
originzifa.loc[(originzifa['账号'].str.contains('amazon')), '平台'] = 'amazon'
originzifa.loc[(originzifa['账号'].str.contains('wayfair')), '平台'] = 'wayfair'
originzifa.loc[(originzifa['账号'].str.contains('ebay')), '平台'] = 'ebay'
originzifa.loc[(originzifa['账号'].str.contains('walmart')), '平台'] = 'walmart'
originzifa.loc[(originzifa['账号'].str.contains('shopify')), '平台'] = 'shopify'

originzifa_wayfair = originzifa[((originzifa['平台']) == 'wayfair')]
originzifa_wayfair = originzifa_wayfair.reset_index(drop=True)
originzifa_wayfair['平台订单号'] = originzifa_wayfair['平台订单号'].str[0:11]

originzifa_walmart = originzifa[((originzifa['平台']) == 'walmart')]
originzifa_walmart = originzifa_walmart.reset_index(drop=True)
originzifa_walmart['平台订单号'] = originzifa_walmart['平台订单号'].str[0:13]

originzifa_shopify = originzifa[((originzifa['平台']) == 'shopify')]
originzifa_shopify = originzifa_shopify.reset_index(drop=True)
originzifa_shopify['平台订单号'] = originzifa_shopify['平台订单号'].str[0:13]

originzifa_amazon = originzifa[((originzifa['平台']) == 'amazon')]
originzifa_amazon = originzifa_amazon.reset_index(drop=True)
originzifa_amazon['平台订单号'] = originzifa_amazon['平台订单号'].str[0:19]

originzifa_ebay = originzifa[((originzifa['平台']) == 'ebay')]
originzifa_ebay = originzifa_ebay.reset_index(drop=True)
originzifa_ebay['平台订单号'] = originzifa_ebay['平台订单号'].str[0:14]

newzifa = pd.concat([originzifa_amazon, originzifa_ebay, originzifa_wayfair, originzifa_walmart, originzifa_shopify])
newzifa = newzifa.reset_index(drop=True)
newzifa.drop('平台', axis=1, inplace=True)

新系统完成
                 平台订单号  账号        平台SKU          公司SKU  数量              包裹号  \
0  111-1984040-5269802  宫本  GB-103483-1   PTG103482-01   1  AK2020122500001   
1  113-9678451-1602655  宫本  G-1015038-1  USAN1015038-1   1  AK2020122500005   

   订单类型 订单状态                订单时间  
0  正常订单  已发货 2020-12-25 03:40:00  
1  正常订单  已发货 2020-12-25 05:57:00  
                 平台订单号          账号            平台SKU          公司SKU  数量  \
0  113-1806850-0610607  Youruisite  U-USAN1011322-6  USAN1011322-6   1   
1  112-0394968-5294664  Youruisite     HR-WH41-NLZA  USAN1009728-1   1   

             包裹号  订单类型 订单状态                 订单时间  
0  AK20030910107  正常订单  已发货  2020-03-10 02:30:14  
1  AK20030213020  正常订单  已发货  2020-03-03 07:00:11  
旧系统完成
                 平台订单号  账号        平台SKU          公司SKU  数量              包裹号  \
0  111-1984040-5269802  宫本  GB-103483-1   PTG103482-01   1  AK2020122500001   
1  113-9678451-1602655  宫本  G-1015038-1  USAN1015038-1   1  AK2020122500005   

   订单类型 订单状态                 订单

In [53]:
inventory = newzifa[newzifa['订单类型'] == '补寄订单']

AdDate = datetime.date(2021,1, 1)
ENDDate = datetime.date(2022,5, 1)
inventory['订单时间'] = pd.to_datetime(inventory['订单时间'])
# inventory['补寄sku订单时间'] = pd.to_datetime(inventory['补寄sku订单时间'])
# inventory['补寄配件订单时间'] = pd.to_datetime(inventory['补寄配件订单时间'])
inventory = inventory[["公司SKU","订单时间","账号",'数量']]
inventory = inventory[(inventory['订单时间'] >= datetime.datetime.combine(AdDate, datetime.datetime.min.time()))&(inventory['订单时间'] < datetime.datetime.combine(ENDDate, datetime.datetime.min.time()))]


In [54]:
inventory['year'] = inventory['订单时间'].dt.year 
inventory['month'] = inventory['订单时间'].dt.month
inventory = inventory.pivot_table(index=['公司SKU','year','month','账号'], values=['数量'], aggfunc="sum")
inventory = inventory.reset_index()


In [44]:
inventory.to_excel(excel_writer="D:/work/invnetorybuji23.xlsx", index=False,encoding='utf-8')